# Setup

In [118]:
import pymysql.cursors
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import CreateTable
from sqlalchemy import text
import getpass

In [119]:
#prompt user to enter MySQL root password
sql_pass = getpass.getpass()
#create connection string and engine to connect to MySQL database
connection_string = 'mysql+pymysql://root:' + sql_pass + '@localhost:3306/civictech'
engine = create_engine(connection_string)

In [120]:
import pandas as pd

# People

### Names

In [121]:
names= pd.read_csv('profiles/names.csv', sep=';')

In [122]:
names.to_sql('names', engine, 'civictech', if_exists='replace', index=False)

47

### Profiles

In [123]:
people=pd.read_csv('profiles/people_main_db.csv', sep=';')
print(list(people.columns))
print(len(list(people.columns)))

['ind_id', 'geoCountryName', 'geoLocationName', 'summary', 'industryName', 'headline', 'experience', 'education', 'languages', 'publications', 'certifications', 'volunteer', 'honors', 'projects', 'experience1', 'experience2', 'experience3', 'experience4', 'experience5', 'education1', 'education2', 'education3', 'languages_over2', 'honors_stated', 'publications_stated', 'volunteer_stated', 'projects_stated', 'certifications_stated', 'languages_stated', 'consulting_roles', 'direction_roles', 'founding_roles']
32


In [124]:
# We may want to drop the columns with too many missing values.
pd.DataFrame(people.isna().sum().sort_values(ascending=False)).head(15)

,0
summary,15
education3,12
education2,6
geoLocationName,4
experience5,3
experience4,2
geoCountryName,2
education1,1
honors_stated,0
languages_over2,0


In [125]:
# and/or drop text columns 
people.dtypes.sort_values()

ind_id                    int64
consulting_roles          int64
languages_stated          int64
certifications_stated     int64
projects_stated           int64
volunteer_stated          int64
publications_stated       int64
honors_stated             int64
languages_over2           int64
direction_roles           int64
founding_roles            int64
volunteer                object
geoCountryName           object
geoLocationName          object
summary                  object
industryName             object
headline                 object
experience               object
education                object
languages                object
certifications           object
education3               object
education1               object
experience5              object
experience4              object
experience3              object
publications             object
experience1              object
projects                 object
honors                   object
education2               object
experien

In [126]:
# for now we are keeping it as is
people.to_sql('people', engine, 'civictech', if_exists='replace', index=False)

47

### Experience

In [127]:
experience=pd.read_csv('profiles/people_experience_db.csv', sep=';')

In [128]:
print(list(experience.columns))
print(len(list(experience.columns)))
print(len(experience))

['ind_id', 'locationName', 'companyName', 'description', 'company', 'title', 'honors', 'startDate_month', 'startDate_year', 'endDate_month', 'endDate_year', 'industry', 'company_empl_low', 'company_empl_high', 'title_direction', 'titleconsulting', 'titlefounder']
17
235


In [129]:
experience.isna().sum().sort_values(ascending=False)
experience.drop(columns='honors', inplace=True)

In [182]:
experience.isna().sum().sort_values(ascending=False)

endDate_month        82
endDate_year         79
locationName         62
description          57
company_empl_low     25
company_empl_high    25
startDate_month      12
companyName           5
title                 5
startDate_year        5
exp_id                0
ind_id                0
industry              0
title_direction       0
titleconsulting       0
titlefounder          0
dtype: int64

In [183]:
experience['startDate_year'] = experience['startDate_year'].fillna(0)
experience['startDate_month'] = experience['startDate_month'].fillna(0)
experience['endDate_year'] = experience['endDate_year'].fillna(0)
experience['endDate_month'] = experience['endDate_month'].fillna(0)
experience['company_empl_low'] = experience['company_empl_low'].fillna(999)
experience['company_empl_high'] = experience['company_empl_high'].fillna(999)

In [185]:
experience['startDate_year'] = experience['startDate_year'].astype(int)
experience['startDate_month'] = experience['startDate_month'].astype(int)
experience['endDate_year'] = experience['endDate_year'].astype(int)
experience['endDate_month'] = experience['endDate_month'].astype(int)
experience['company_empl_low'] = experience['company_empl_low'].astype(int)
experience['company_empl_high'] = experience['company_empl_high'].astype(int)

In [186]:
experience.dtypes.sort_values()

exp_id                int64
ind_id                int64
startDate_month       int64
startDate_year        int64
endDate_month         int64
endDate_year          int64
company_empl_low      int64
company_empl_high     int64
title_direction       int64
titleconsulting       int64
titlefounder          int64
locationName         object
companyName          object
description          object
title                object
industry             object
dtype: object

In [131]:
experience.drop(columns='company', inplace=True)

In [187]:
experience.industry[1]

['Management Consulting']

In [133]:
import re
def clean_industry(x):
    x=str(x)
    pattern = "[a-zA-Z]+ [a-zA-Z]+|[a-zA-Z]+"
    return re.findall(pattern, x)

In [134]:
experience['industry']= experience['industry'].apply(clean_industry)

In [188]:
experience['industry'].value_counts()

[Computer Software]                       55
[Information Technology, and Services]    45
[Civic, Social Organization]              35
[Management Consulting]                   20
[nan]                                     20
[Higher Education]                        20
[Internet]                                10
[Venture Capital, Private Equity]          5
[Nonprofit Organization, Management]       5
[Graphic Design]                           5
[Performing Arts]                          5
[Marketing and, Advertising]               5
[Research]                                 5
Name: industry, dtype: int64

In [140]:
experience.reset_index(inplace=True)
experience.rename(columns={"index":"exp_id"}, inplace=True)

In [190]:
experience.reset_index(inplace=True)

In [196]:
experience.drop(columns='index')
experience.shape

(235, 17)

In [197]:
experience.to_csv("profiles/experience2.csv", sep=';', index=False)

In [191]:
experience.to_sql('experience', engine, 'civictech', if_exists='replace', index=False)
# specify data types 

OperationalError: (pymysql.err.OperationalError) (1241, 'Operand should contain 1 column(s)')
[SQL: INSERT INTO civictech.experience (`index`, exp_id, ind_id, `locationName`, `companyName`, description, title, `startDate_month`, `startDate_year`, `endDate_month`, `endDate_year`, industry, company_empl_low, company_empl_high, title_direction, titleconsulting, titlefounder) VALUES (%(index)s, %(exp_id)s, %(ind_id)s, %(locationName)s, %(companyName)s, %(description)s, %(title)s, %(startDate_month)s, %(startDate_year)s, %(endDate_month)s, %(endDate_year)s, %(industry)s, %(company_empl_low)s, %(company_empl_high)s, %(title_direction)s, %(titleconsulting)s, %(titlefounder)s)]
[parameters: ({'index': 0, 'exp_id': 0, 'ind_id': 0, 'locationName': 'Paris Metropolitan Region', 'companyName': 'bluenove', 'description': "Bluenove accompagne la transformation positive des organisations en faisant levier de l'intelligence collective de toutes leurs parties prenantes.\nPlus d'informations sur www.blue

### Education

In [145]:
education=pd.read_csv('profiles/people_education_db.csv', sep=';')

In [146]:
print(list(education.columns))
print(len(list(education.columns)))
print(len(education))

['ind_id', 'school', 'degreeName', 'schoolName', 'fieldOfStudy', 'startDate_year']
6
141


In [147]:
education.isna().sum().sort_values(ascending=False)

fieldOfStudy      51
school            46
degreeName        44
startDate_year    30
schoolName        19
ind_id             0
dtype: int64

In [178]:
education.dtypes.sort_values(ascending=False)

school             object
degreeName         object
schoolName         object
fieldOfStudy       object
startDate_year    float64
ind_id              int64
dtype: object

In [177]:
education.startDate_year.value_counts(dropna=False)
education['startDate_year'] = education['startDate_year'].fillna(0)

In [180]:
education['startDate_year']= education['startDate_year'].astype(int)

In [181]:
education.to_sql('education', engine, 'civictech', if_exists='replace', index=False)

141

## Companies finance data

In [95]:
companies_finance=pd.read_csv('scraping_organizations/companies_finance_data.csv', sep=';') # possible index drop

In [96]:
companies_finance.head()

,Unnamed: 0,Chiffre d'affaires (€),Marge brute (€),EBITDA - EBE (€),Résultat d'exploitation (€),Résultat net (€),Croissance,Taux de croissance du CA (%),Taux de marge brute (%),Taux de marge d'EBITDA (%),...,Marge nette (%),Rentabilité sur fonds propres (%),Rentabilité économique (%),Valeur ajoutée (€),Valeur ajoutée / CA (%),Structure d'activité,Salaires et charges sociales (€),Salaires / CA (%),Impôts et taxes (€),Chiffre d'affaires à l'export (€)
0,2017_citility,NaN,NaN,NaN,NaN,-562K,2017.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2017.0,NaN,NaN,NaN,NaN
1,2016_citility,"30,7K",527K,-295K,-296K,-238K,2016.0,4,"1,72K",-961,...,-775,-623,-173,351K,"1,14K",2016.0,636K,"2,07K","9,16K",NaN
2,2019_voxcracy,"46,3K","46,3K","-31,5K","-38,5K",-39K,2019.0,177,100,-682,...,-842,-76,-63,"2,17K",47,2019.0,"33,6K",725,137,NaN
3,2018_voxcracy,"16,7K",157K,-123K,-130K,-112K,2018.0,861,940,-734,...,-668,-202,-169,-21K,-126,2018.0,101K,602,"1,12K",5K
4,2017_voxcracy,"1,74K","75,9K","-24,4K","-30,3K","-23,4K",2017.0,NaN,"4,36K","-1,4K",...,"-1,35K",-41,-39,"-5,04K",-290,2017.0,"19,1K","1,1K",329,NaN


In [97]:
companies_finance.to_sql('companies_finance', engine, 'civictech', if_exists='replace', index=False)

74

### Companies info

In [98]:
companies_info=pd.read_csv('scraping_organizations/companies_info_data.csv', sep=';')
print(companies_info.columns)

Index(['Unnamed: 0', 'Adresse :', 'Activité :', 'Effectif :', 'Création :',
       'Dirigeants :', 'Forme juridique :', 'Inscription au RCS :',
       'Capital social :', 'Activité principale déclarée :',
       'Code NAF ou APE :', 'Domaine d’activité :', 'Dirigeant :',
       'Inscription au RNA :', 'Identifiant association :',
       'Objet de l'association :', 'Statut INSEE :'],
      dtype='object')


In [99]:
new_cols=[]
for i in list(companies_info.columns):
    i=i.lower().replace(' :', '').replace('é', 'e').replace("(", '').replace(')', '').replace(' ', '_').replace("'",'').replace("’",'').replace("è", "e")
    new_cols.append(i)
print(new_cols)

['unnamed:_0', 'adresse', 'activite', 'effectif', 'creation', 'dirigeants', 'forme_juridique', 'inscription_au_rcs', 'capital_social', 'activite_principale_declaree', 'code_naf_ou_ape', 'domaine_dactivite', 'dirigeant', 'inscription_au_rna', 'identifiant_association', 'objet_de_lassociation', 'statut_insee']


In [100]:
companies_info.columns=new_cols
companies_info.head()

,unnamed:_0,adresse,activite,effectif,creation,dirigeants,forme_juridique,inscription_au_rcs,capital_social,activite_principale_declaree,code_naf_ou_ape,domaine_dactivite,dirigeant,inscription_au_rna,identifiant_association,objet_de_lassociation,statut_insee
0,citility,5 RUE DE LA CLAIRE 69009 LYON 9EME,Édition de logiciels applicatifs,0 salarié (donnée 2019),05/05/2014,"André MAY, ODICEO, Sabine SCHNECK","SAS, société par actions simplifiée","INSCRIT (au greffe de LYON, le 12/05/2014)","39 620,00 €",Edition de logiciels applicatifs.,58.29C (Édition de logiciels applicatifs),Édition,NaN,NaN,NaN,NaN,NaN
1,poligma,RPT BENJAMIN FRANKLIN 34960 MONTPELLIER CEDEX 2,Conseil en systèmes et logiciels informatiques,Entre 3 et 5 salariés (donnée 2020),01/09/2015,NaN,"SAS, société par actions simplifiée","INSCRIT (au greffe de MONTPELLIER, le 11/09/2015)","20 944,00 €",Développement de services informatiques et num...,62.02A (Conseil en systèmes et logiciels infor...,"Programmation, conseil et autres activités inf...",Philippe GERARD,NaN,NaN,NaN,NaN
2,voxcracy,1133 RTE FENERIE 06580 PEGOMAS,Programmation informatique,0 salarié,01/09/2014,"Olivier ROCCA, Pascal RUSCICA","SAS, société par actions simplifiée","INSCRIT (au greffe de GRASSE, le 17/07/2014)","1 000,00 €","La recherche, le développement et la commercia...",62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",NaN,NaN,NaN,NaN,NaN
3,LLL_2,16 RUE DU CAIRE 75002 PARIS 2,Autres activités de soutien aux entreprises n....,Entre 3 et 5 salariés (donnée 2020),16/01/2015,"ANNAMAMASHOW, Raymond Maeder, 3APEXCO","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 03/02/2015)","45 000,00 €",Développement de nouvelles formes de collabora...,82.99Z (Autres activités de soutien aux entrep...,Activités administratives et autres activités ...,NaN,NaN,NaN,NaN,NaN
4,bluenove,112 B RUE CARDINET 75017 PARIS 17,Conseil pour les affaires et autres conseils d...,Entre 20 et 49 salariés (donnée 2020),23/01/2008,"GROUPE BLUENOVE INC., Guillaume Drancy, Carole...","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 21/11/2019)","56 445,00 €",L'activité de conseil en stratégie d'accompagn...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,NaN,NaN,NaN,NaN,NaN


In [101]:
companies_info

,unnamed:_0,adresse,activite,effectif,creation,dirigeants,forme_juridique,inscription_au_rcs,capital_social,activite_principale_declaree,code_naf_ou_ape,domaine_dactivite,dirigeant,inscription_au_rna,identifiant_association,objet_de_lassociation,statut_insee
0,citility,5 RUE DE LA CLAIRE 69009 LYON 9EME,Édition de logiciels applicatifs,0 salarié (donnée 2019),05/05/2014,"André MAY, ODICEO, Sabine SCHNECK","SAS, société par actions simplifiée","INSCRIT (au greffe de LYON, le 12/05/2014)","39 620,00 €",Edition de logiciels applicatifs.,58.29C (Édition de logiciels applicatifs),Édition,NaN,NaN,NaN,NaN,NaN
1,poligma,RPT BENJAMIN FRANKLIN 34960 MONTPELLIER CEDEX 2,Conseil en systèmes et logiciels informatiques,Entre 3 et 5 salariés (donnée 2020),01/09/2015,NaN,"SAS, société par actions simplifiée","INSCRIT (au greffe de MONTPELLIER, le 11/09/2015)","20 944,00 €",Développement de services informatiques et num...,62.02A (Conseil en systèmes et logiciels infor...,"Programmation, conseil et autres activités inf...",Philippe GERARD,NaN,NaN,NaN,NaN
2,voxcracy,1133 RTE FENERIE 06580 PEGOMAS,Programmation informatique,0 salarié,01/09/2014,"Olivier ROCCA, Pascal RUSCICA","SAS, société par actions simplifiée","INSCRIT (au greffe de GRASSE, le 17/07/2014)","1 000,00 €","La recherche, le développement et la commercia...",62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",NaN,NaN,NaN,NaN,NaN
3,LLL_2,16 RUE DU CAIRE 75002 PARIS 2,Autres activités de soutien aux entreprises n....,Entre 3 et 5 salariés (donnée 2020),16/01/2015,"ANNAMAMASHOW, Raymond Maeder, 3APEXCO","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 03/02/2015)","45 000,00 €",Développement de nouvelles formes de collabora...,82.99Z (Autres activités de soutien aux entrep...,Activités administratives et autres activités ...,NaN,NaN,NaN,NaN,NaN
4,bluenove,112 B RUE CARDINET 75017 PARIS 17,Conseil pour les affaires et autres conseils d...,Entre 20 et 49 salariés (donnée 2020),23/01/2008,"GROUPE BLUENOVE INC., Guillaume Drancy, Carole...","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 21/11/2019)","56 445,00 €",L'activité de conseil en stratégie d'accompagn...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,NaN,NaN,NaN,NaN,NaN
5,ecivis,3 IMP JEAN MERMOZ 31470 FONSORBES,Programmation informatique,0 salarié (donnée 2018),16/12/2016,NaN,"SAS, société par actions simplifiée","RADIÉ (du greffe de TOULOUSE, le 31/10/2018)","20 000,00 €",Prestation de services et accès à une platefor...,62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",Rémi POSTIC,NaN,NaN,NaN,NaN
6,make4,14 RUE ST GUILLAUME 75007 PARIS 7,Autres organisations fonctionnant par adhésion...,Entre 3 et 5 salariés (donnée 2020),11/01/2017,NaN,Association déclarée,Non inscrit,NaN,NaN,94.99Z (Autres organisations fonctionnant par ...,Activités des organisations associatives,NaN,NaN,NaN,NaN,NaN
7,civimetric,12 PAS DE LA FONDERIE 75011 PARIS 11,Conseil pour les affaires et autres conseils d...,0 salarié (donnée 2019),16/02/2018,NaN,"SARL, société à responsabilité limitée","RADIÉ (du greffe de PARIS, le 04/08/2020)","3 300,00 €",Le conseil et la réalisation d'études quantita...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,Fabien Durand,NaN,NaN,NaN,NaN
8,voteetvous,133 RUE ST DOMINIQUE 75007 PARIS 7,Autres organisations fonctionnant par adhésion...,Au moins 1 salarié (donnée 2023),27/01/2014,NaN,Association déclarée,Non inscrit,NaN,NaN,94.99Z (Autres organisations fonctionnant par ...,Activités des organisations associatives,NaN,INSCRITE le 27/01/2014,W751223032,"Renforcer l'exercice démocratique du vote, en ...",NaN
9,vooter,71 RUE D AGUESSEAU 92100 BOULOGNE-BILLANCOURT,Programmation informatique,0 salarié (donnée 2023),23/04/2015,"Stephane BEQUIN, Dimitri DELATTRE","SAS, société par actions simplifiée","INSCRIT (

In [102]:
companies_info['creation'] = pd.to_datetime(companies_info['creation'], format="%d/%m/%Y")

In [103]:
list(companies_info['unnamed:_0'])

['citility',
 'poligma',
 'voxcracy',
 'LLL_2',
 'bluenove',
 'ecivis',
 'make4',
 'civimetric',
 'voteetvous',
 'vooter',
 'organigram_97',
 'forcity_mere',
 'mymairie',
 'quorum',
 'fullmobs',
 'mairesetcitoyens',
 'parlonspo',
 'make3',
 'consultvox',
 'loomio',
 'politiker',
 'citizenlabbv',
 'datagora',
 'ouiville',
 'forcity',
 'accromedias',
 'lesbricodeurs',
 'crowdpac',
 'ledrenche',
 'youvote',
 'make2',
 'ecreall_nova_ideo',
 'neocity',
 'citieszen',
 'politicus',
 'capcollectif',
 'whip',
 'voxestudio',
 'democracyos',
 'polipart',
 'LLL',
 'hacktiv',
 'voxpublic',
 'citybay_ambitions',
 'politizr',
 'digitalebox',
 'mesopinions',
 'assemblee_virtuelle',
 'madeinvote',
 'wesignit',
 'mieuxvoter',
 'kawaa',
 'ciwik',
 'data_for_good',
 'digiworks',
 'equivote_myopencity',
 'make',
 'kohero',
 'openagora',
 'moneparti',
 'nousrassemble',
 'bittle',
 'marston_massiet',
 'democratieouverte',
 'koom',
 'purpoz',
 'reperage_urbain',
 'regardscitoyens',
 'D21',
 'IDcity',
 'Stig',

In [104]:
companies_info.rename(columns={"unnamed:_0": "name"}, inplace=True)

In [105]:
companies_info.to_sql('companies_info', engine, 'civictech', if_exists='replace', index=False)

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
### you can pass data types here 
from sqlalchemy.types import Integer
df.to_sql('integers', con=engine, index=False,
          dtype={"A": Integer()})

76